In [1]:
import time

import pinocchio
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [13]:
import numpy as np
from tracikpy import TracIKSolver
import pinocchio

ee_pose = np.array([[1, 0, 1, 0.3],
                    [0, 1, 0, 0.2],
                    [0, 0, 1, 0.1],
                    [ 0.,  0.,  0., 1.]])

urdf_file = "panda_description/urdf/panda.urdf"
ee_frame = 'panda_link7'

ik_solver = TracIKSolver(
    urdf_file,
    "base_link",
    ee_frame,
)
qout = ik_solver.ik(ee_pose, qinit=np.zeros(ik_solver.number_of_joints))
print(qout)
ik_solver.fk(qout)

[ 2.32247581  1.76275288 -1.03213629 -2.54752572 -0.86755672  1.92788229
  0.45118279]


[ WARN] [1714268187.467200653]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.


array([[ 0.91402293, -0.08454572,  0.39675447,  0.29999701],
       [ 0.2053586 ,  0.93989514, -0.27280942,  0.19999951],
       [-0.34984273,  0.330831  ,  0.87644789,  0.10000631],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
from src import damped_IK

urdf_file = "panda_description/urdf/panda.urdf"
# urdf_file = '../mona_torso_description/urdf/mona_torso.urdf'

constraints = [{'frame': "panda_joint7", 'type': 'position', 'weigth':1},
               {'frame': "panda_joint7", 'type': 'orientation', 'weigth':1},
            #    {'frame': "Revolute 28", 'type': 'position', 'weigth':1},
            #    {'frame': "Revolute 28", 'type': 'orientation', 'weigth':1},
               ]

IK_solver = damped_IK(urdf_file, constraints, verbose=True, dt=0.1, max_iteration=1000, damp=1e-12)

q0 = pinocchio.randomConfiguration(IK_solver.model)
# q0=np.zeros(9)

targets = [
            np.array([0.5, 0, 0.4]),
            np.array([[0,1,0], [1,0,0], [0,0,-1]]),
            # np.array([-0.4, 0.2, 0.1]),
            # np.array([[0,1,0], [1,0,0], [0,0,-1]]),
            ]

t0 = time.time()
q_path, success = IK_solver.run(q0, targets, return_path=True)
print(f"Solver successful: {success} in {1000*(time.time()-t0):.2f}ms with {len(q_path)} iterations")

print(q_path[0])

-----
Position constraint of joint  7
Reached:  [ 0.498 -0.001  0.401]
Target: [0.5 0.  0.4]
-----
Orientation constraint of joint  7
Reached:
 [[-0.001  1.    -0.001]
 [ 1.     0.001  0.   ]
 [ 0.    -0.001 -1.   ]]
Target:
 [[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
Solver successful: True in 24.12ms with 157 iterations
[-1.352  0.14  -0.723 -0.79   0.073  2.5    0.183  0.002  0.018]


In [8]:
from src import croco_IK

urdf_file = "panda_description/urdf/panda.urdf"
# urdf_file = '../mona_torso_description/urdf/mona_torso.urdf'

constraints = [{'frame': "panda_link8", 'type': 'position', 'weigth':100},
               {'frame': "panda_link8", 'type': 'orientation', 'weigth':100},
            #    {'frame': "Revolute 28", 'type': 'position', 'weigth':1},
            #    {'frame': "Revolute 28", 'type': 'orientation', 'weigth':1},
               ]

IK_solver = croco_IK(urdf_file, constraints, verbose=True, dt=1e-3, max_iteration=20, damp=1e-12)

q0 = pinocchio.randomConfiguration(IK_solver.model)
# q0=np.zeros(9)

targets = [
            np.array([0.5, 0, 0.4]),
            np.array([[0,1,0], [1,0,0], [0,0,-1]]),
            # np.array([-0.4, 0.2, 0.1]),
            # np.array([[0,1,0], [1,0,0], [0,0,-1]]),
            ]

t0 = time.time()
q_path, success = IK_solver.run(q0, targets, return_path=True)
print(f"Solver successful: {success} in {1000*(time.time()-t0):.2f}ms with {len(q_path)} iterations")

print(q_path[-1])

-----
Position constraint of joint  panda_link8
Reached:  [-0.51   0.039  0.686]
Target: [0.5 0.  0.4]
-----
Orientation constraint of joint  panda_link8
Reached:
 [[ 0.601 -0.729 -0.327]
 [ 0.713  0.304  0.632]
 [-0.361 -0.613  0.703]]
Target:
 [[ 0  1  0]
 [ 1  0  0]
 [ 0  0 -1]]
Solver successful: True in 158.58ms with 21 iterations
[ -578.865  -113.567 -1092.108   924.831  -715.915  1465.831   544.084
    91.479  -202.514]


In [93]:
q_0 = (IK_solver.model.upperPositionLimit-IK_solver.model.lowerPositionLimit)/2
# q_path = np.zeros(12)

q_path = np.vstack((np.linspace(q0, IK_solver.model.lowerPositionLimit, 50), 
                    np.linspace(IK_solver.model.lowerPositionLimit, IK_solver.model.lowerPositionLimit, 500), 
                    np.linspace(IK_solver.model.lowerPositionLimit, IK_solver.model.upperPositionLimit, 100),
                    np.linspace(IK_solver.model.upperPositionLimit, IK_solver.model.upperPositionLimit, 500),
                    np.linspace(IK_solver.model.upperPositionLimit, q0, 50)))

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

q_path = np.array(q_path)
fig = make_subplots(rows=q_path.shape[1], cols=1)
for i in range(q_path.shape[1]):
    fig.add_trace(go.Scatter(y=q_path[:,i]), row=i+1, col=1)
    fig.add_hline(y=IK_solver.model.lowerPositionLimit[i], row=i+1, col=1, line_dash='dash')
    fig.add_hline(y=IK_solver.model.upperPositionLimit[i], row=i+1, col=1, line_dash='dash')

fig.update_layout(height=2000, showlegend=False)
fig.show()

In [4]:
import pybullet as p
import time
import pybullet_data

try: p.disconnect()
except: pass


physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
robotSim = p.loadURDF(urdf_file, [0,0,0], p.getQuaternionFromEuler([0,0,0]))
p.resetDebugVisualizerCamera(cameraYaw = 90, cameraPitch= -20,cameraDistance = 1,cameraTargetPosition=[0.7,-0.2,0.7])

# Show target as red sphere
for constraint, target in zip(constraints, targets):
    if constraint['type'] == 'position':
        test_visual = p.createVisualShape(p.GEOM_SPHERE, radius=0.1, rgbaColor=[1,0,0,0.5])
        test_body = p.createMultiBody(baseMass=0, baseVisualShapeIndex=test_visual, basePosition = target)

# Replay robot trajectory to target     
for i, q in enumerate(q_path):
    for jointIndex in range(len(q)):
        p.resetJointState(robotSim, jointIndex, q[jointIndex])
        p.setJointMotorControl2(robotSim, jointIndex, p.POSITION_CONTROL,
                                targetPosition=q[jointIndex], force=10)
    if i==0: time.sleep(1)
    time.sleep(0.1)

time.sleep(2)
p.disconnect()

pybullet build time: Oct 14 2023 15:43:53


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 11.0.1, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 20.3.5
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 20.3.5
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 11.0.1, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroy

In [89]:
p.disconnect()

numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
